In [1]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas

# import modules
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import datetime
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Combining dataframes

In [2]:
dkpol = pd.read_csv('dkpol_data.csv').reset_index(drop=True)

dkpol

,Unnamed: 0,username,description,location,following,followers,totaltweets,retweetcount,text,hashtags,date
0,0,KKockii,Indfødsretnørd. 🇩🇰\nFT-kandidat for @alterna...,"Stevns, Danmark",301,272,1145,16,Generation Identitær påstod forleden i Informa...,[],2021-08-19 08:59:20
1,1,MargueritteEch,|⚜️Author & Freelance-writer⚜️|L&R~Egmont~Alle...,⚜️🇩🇰Denmark~Copenhagen Area,2758,3251,32740,0,@Enhedslisten @regeringDK Når stat modtager fl...,['dkpol'],2021-08-19 08:58:27
2,2,dkpol_net,"https://t.co/RXPQSSsMaN er et website, der min...",Danmark,73,2889,23044,0,Lige nu er @KristinMClement populær på #dkpol ...,['dkpol'],2021-08-19 08:58:05
3,3,kristeligt,Avisen om livets store spørgsmål. Og især #dkt...,Denmark,4407,20973,27829,0,.@Sygeplejeraadet og @regionerne kan fortsat i...,['dkpol'],2021-08-19 08:57:39
4,4,HermansenBo,I work for the Danish State railways. I ...,"Aarhus, Danmark",128,21,838,9,Forbud mod Croptops = negativ social kontrol -...,[],2021-08-19 08:57:06
...,...,...,...,...,...,...,...,...,...,...,...
1990,280,tbhdkx,Privatlivsnørd og en del af @cryptohagen (#cry...,Danmark,424,1182,48175,1,Dagens tegning: Klæder skaber folk\n\nPernille...,['dkpol'],2021-08-13 18:02:57
1991,281,SFpolitik,Socialistisk Folkeparti,Denmark,291,30402,10848,6,Har indlæg i JP i dag om mine tanker for krimi...,[],2021-08-13 18:01:33
1992,282,KennethNrgaard,Lidt af hvert om alt muligt,Denmark,828,1797,52137,1,Dagens tegning: Klæder skaber folk\n\nPernille...,['dkpol'],2021-08-13 18:01:07
1993,283,DOppenheuser,Far til 3 skønne unger. Er her primært for lid...,"København, Danmark",312,48,336,11,En svinebonde får 9 kroner for et kilo bulk gr...,[],2021-08-13 17:59:04


### Cleaning the DataFrame

#### Dropping & Creating Columns 

In [3]:
#Dropping collumns
data_set1 = dkpol.drop(['Unnamed: 0','username', 'description', 'followers', 'following'], axis=1).reset_index(drop = True)

#Creating collumn for hashtags
data_set1['hashtags'] = dkpol['text'].apply(lambda x: re.findall(r'#(\w+)', x))

#Calling text column
tweets = data_set1['text'].values

#### Emojis & Emoticons

In [4]:
#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

In [7]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
         "]+", flags=re.UNICODE)

#### Cleaning with Regex Expressions

In [71]:
import numpy as np
wnl = nltk.WordNetLemmatizer()
stopWords = set(stopwords.words('danish'))

def clean_tweet(tweet):
    if type(tweet) == np.float64:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) 
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('[.,*?:]',' ', temp)
    temp = re.sub('\v', ' ', temp)
    temp = re.sub("[\\n]",' ', temp)
    temp = re.sub("“",' ', temp)
    temp = re.sub("”",' ', temp)
    temp = re.sub('1/n',' ', temp)
    temp = re.sub('2/n',' ', temp)
    temp = re.sub('3/n',' ', temp)
    temp = re.sub('4/n',' ', temp)
    temp = re.sub('5/n',' ', temp)
    temp = re.sub('6/n',' ', temp)
    temp = re.sub('7/n',' ', temp)
    temp = re.sub('8/n',' ', temp)
    temp = re.sub('-','', temp) 
    temp = re.sub('%','', temp) 
    temp = re.sub('["]','', temp)
    temp = re.sub("[']","", temp)
    temp = re.sub('[0-9]', ' ', temp)
    temp = re.sub('\+', ' ', temp)
   
    temp = emoji_pattern.sub(r'', temp)
    return temp

#### Importing Dictionary Afinn (Danish)

In [5]:
#importer dansk som sprog
#pip install afinn
from afinn import Afinn
afinn = Afinn(language='da')

#### Looping over rows in "text" column: cleaning and assigning sentiment score

In [72]:

tweetser = [None for i in range(len(data_set1))]

score = [None for i in range(len(data_set1))]
for i in range(len(data_set1)):  
    tweetser[i] = clean_tweet(data_set1.text.values[i])
    score[i]= afinn.score(tweetser[i])
    
#count reetweets
#Counter(tweetser).most_common()

#### Inserting new columns into the original data set

In [73]:
data_set1['Clean Text'] = tweetser
data_set1['Sentiment'] = score
dkpol_final = data_set1.drop(['text'], axis=1).reset_index(drop = True)
dkpol_final

,location,totaltweets,retweetcount,hashtags,date,Clean Text,Sentiment
0,"Stevns, Danmark",1145,16,[dkpol],2021-08-19 08:59:20,generation identitær påstod forleden i informa...,0.0
1,⚜️🇩🇰Denmark~Copenhagen Area,32740,0,[dkpol],2021-08-19 08:58:27,når stat modtager flygtninger bærer den et s...,0.0
2,Danmark,23044,0,[dkpol],2021-08-19 08:58:05,lige nu er populær på,3.0
3,Denmark,27829,0,[dkpol],2021-08-19 08:57:39,og kan fortsat ikke blive enige politikern...,-3.0
4,"Aarhus, Danmark",838,9,"[croptop, dkmedier, dkpol]",2021-08-19 08:57:06,forbud mod croptops = negativ social kontrol ...,-4.0
...,...,...,...,...,...,...,...
1990,Danmark,48175,1,[dkpol],2021-08-13 18:02:57,dagens tegning klæder skaber folk pernille v...,0.0
1991,Denmark,10848,6,"[dkpol, dkkrim]",2021-08-13 18:01:33,har indlæg i jp i dag om mine tanker for krimi...,1.0
1992,Denmark,52137,1,[dkpol],2021-08-13 18:01:07,dagens tegning klæder skaber folk pernille v...,0.0
1993,"København, Danmark",336,11,"[dkpol, dkgreen]",2021-08-13 17:59:04,en svinebonde får kroner for et kilo bulk gr...,0.0


### Term Frequency 

#### Deleting ' , in strings

In [74]:
clean_data = [clean_tweet(tweets) for tweets in tweets]

c = str(clean_data) #converting the clean_data from a list to a string
a = c.replace("'", "")
b = a.replace(",", "")

#### Removing stopwords

In [75]:
#Remove danish stopWords
stopWords = set(stopwords.words('danish'))
sent_sw_removed = [i for i in nltk.word_tokenize(b.lower()) if i not in stopWords]

#### Counting most frequent words

In [76]:
from collections import Counter

tæl = Counter(sent_sw_removed).most_common()

#### Ordering in tabel

In [77]:
counter_tabel=pd.DataFrame(tæl)
counter_tabel.columns = ['Words', 'N']

counter_tabel

,Words,N
0,så,390
1,kan,373
2,afghanistan,228
3,vores,151
4,få,142
...,...,...
7184,straffesagsomkostninger,1
7185,udslusningspladser,1
7186,krigene,1
7187,dominerede,1
